<a href="https://colab.research.google.com/github/AMAYadav/PROJECTS-/blob/main/market_risk_02_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [263]:
!pip3 install yfinance -- quiet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement quiet (from versions: none)
ERROR: No matching distribution found for quiet


In [264]:
# ---------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# ----------------
import yfinance as yf

In [265]:
url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=2022"
data= pd.read_html(url,)[0]

data.head()

Date  20 YR  30 YR  Extrapolation Factor  8 WEEKS BANK DISCOUNT  \
0  01/03/2022    NaN    NaN                   NaN                    NaN   
1  01/04/2022    NaN    NaN                   NaN                    NaN   
2  01/05/2022    NaN    NaN                   NaN                    NaN   
3  01/06/2022    NaN    NaN                   NaN                    NaN   
4  01/07/2022    NaN    NaN                   NaN                    NaN   

   COUPON EQUIVALENT  17 WEEKS BANK DISCOUNT  COUPON EQUIVALENT.1  \
0                NaN                     NaN                  NaN   
1                NaN                     NaN                  NaN   
2                NaN                     NaN                  NaN   
3                NaN                     NaN                  NaN   
4                NaN                     NaN                  NaN   

   52 WEEKS BANK DISCOUNT  COUPON EQUIVALENT.2  ...  4 Mo  6 Mo  1 Yr  2 Yr  \
0                     NaN                  NaN  ...   NaN  0.22  0.40  0.78   
1                     NaN                  NaN  ...   NaN  0.22  0.38  0.77   
2                     NaN                  NaN  ...   NaN  0.22  0.41  0.83   
3                     NaN                  NaN  ...   NaN  0.23  0.45  0.88   
4                     NaN                  NaN  ...   NaN  0.24  0.43  0.87   

   3 Yr  5 Yr  7 Yr  10 Yr  20 Yr  30 Yr  
0  1.04  1.37  1.55   1.63   2.05   2.01  
1  1.02  1.37  1.57   1.66   2.10   2.07  
2  1.10  1.43  1.62   1.71   2.12   2.09  
3  1.15  1.47  1.66   1.73   2.12   2.09  
4  1.17  1.50  1.69   1.76   2.15   2.11  

[5 rows x 23 columns]

In [266]:
# data[['Date', '1 Mo', '2 Mo', '3 Mo', '6 Mo', '1 Yr', '2 Yr', '3 Yr', '5 Yr','7 Yr', '10 Yr', '20 Yr', '30 Yr']]
df=data[['Date', '6 Mo', '1 Yr', '2 Yr', '3 Yr', '5 Yr','7 Yr',]].set_index('Date')
df.tail()

6 Mo  1 Yr  2 Yr  3 Yr  5 Yr  7 Yr
Date                                          
12/19/2022  4.71  4.64  4.25  3.99  3.70  3.67
12/20/2022  4.70  4.64  4.25  4.03  3.79  3.78
12/21/2022  4.67  4.60  4.21  4.00  3.78  3.77
12/22/2022  4.66  4.64  4.24  4.02  3.79  3.77
12/23/2022  4.67  4.66  4.31  4.09  3.86  3.83

In [267]:
df.index = pd.to_datetime(df.index)

In [268]:
plt.ylabel(f'rates', fontsize=10)
plt.xlabel(f'tenors', fontsize=10)
fig=sns.lineplot(x=['6 Mo', '1 Yr', '2 Yr', '3 Yr', '5 Yr', '7 Yr', '10 Yr', '20 Yr',
       '30 Yr'],y=[4.67, 4.66, 4.31, 4.09, 3.86, 3.83, 3.75, 3.99, 3.82],).set_title(f'Term Structure for 12/23/2022')

In [269]:
df.describe()[1:]*100

6 Mo        1 Yr        2 Yr        3 Yr        5 Yr        7 Yr
mean  247.534694  276.502041  296.400000  302.963265  298.371429  298.987755
std   151.867702  138.084692  110.450582   98.099788   82.600490   75.911482
min    22.000000   38.000000   77.000000  102.000000  137.000000  155.000000
25%   108.000000  169.000000  235.000000  255.000000  255.000000  252.000000
50%   255.000000  288.000000  303.000000  302.000000  298.000000  299.000000
75%   387.000000  405.000000  411.000000  396.000000  367.000000  361.000000
max   478.000000  480.000000  472.000000  466.000000  445.000000  436.000000

In [270]:
vix=yf.download('^VIX',start=df.index[0],end=df.index[-1],progress=False)[['Adj Close']].round(2)
vix.rename(columns={'Adj Close': 'vix_price'}, inplace=True)
vix_returns=np.log(vix/vix.shift(1))
vix_returns.rename(columns={'Adj Close': 'vix_returns'}, inplace=True)
vix_returns.head()

vix_price
Date                 
2022-01-03        NaN
2022-01-04   0.018502
2022-01-05   0.154235
2022-01-06  -0.006101
2022-01-07  -0.044313

In [271]:
nasdaq=yf.download('^IXIC',start=df.index[0],end=df.index[-1],progress=False)[['Adj Close']].round(2)
nasdaq.rename(columns={'Adj Close': 'index_price'}, inplace=True)
nasdaq_returns=np.log(nasdaq/nasdaq.shift(1))
nasdaq_returns.rename(columns={'Adj Close': 'index_returns'}, inplace=True)
nasdaq_returns.head()

index_price
Date                   
2022-01-03          NaN
2022-01-04    -0.013357
2022-01-05    -0.034020
2022-01-06    -0.001280
2022-01-07    -0.009659

In [272]:
from operator import truediv
final=pd.merge(nasdaq,vix,on='Date').merge(df,how='left',on='Date').dropna()
last_traded_price=final.iloc[[-1]]
last_traded_price

index_price  vix_price  6 Mo  1 Yr  2 Yr  3 Yr  5 Yr  7 Yr
Date                                                                  
2022-12-22     10476.12      21.97  4.66  4.64  4.24  4.02  3.79  3.77

In [273]:
final_returns=pd.merge(nasdaq_returns,vix_returns,on='Date').merge(np.log(df/df.shift(1)),how='left',on='Date').dropna()

volatility=pd.DataFrame(np.std(final_returns)*100,columns=['volatility']).round(2).T
volatility.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, volatility to volatility
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index_price  1 non-null      float64
 1   vix_price    1 non-null      float64
 2   6 Mo         1 non-null      float64
 3   1 Yr         1 non-null      float64
 4   2 Yr         1 non-null      float64
 5   3 Yr         1 non-null      float64
 6   5 Yr         1 non-null      float64
 7   7 Yr         1 non-null      float64
dtypes: float64(8)
memory usage: 180.0+ bytes


In [274]:
pd.concat([volatility,last_traded_price])

index_price  vix_price  6 Mo  1 Yr  2 Yr  3 Yr  5 Yr  \
volatility                  2.03       6.76  4.19  3.80  3.33  3.21  3.05   
2022-12-22 00:00:00     10476.12      21.97  4.66  4.64  4.24  4.02  3.79   

                     7 Yr  
volatility           2.92  
2022-12-22 00:00:00  3.77